In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"

loader = PyMuPDFLoader(pdf_path, extract_images=True)

docs = loader.load()

# 문서 정보 출력
for idx, doc in enumerate(docs):
    print(f"Page {idx + 1}:")
    print("Text:", doc.page_content)  # 페이지의 텍스트 출력
    if "images" in doc.metadata:
        print("Images:", doc.metadata["images"])  # 이미지 정보 출력
    print("-" * 50)


In [ ]:
from dotenv import load_dotenv
load_dotenv('../../.env')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# PDF 파일 로드 및 페이지 분할
loader = PyPDFLoader("C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf")  # PDF 파일 경로 설정
pages = loader.load_and_split()  # 페이지별로 텍스트 추출

# **2. 벡터 스토어 생성**
vectorstore = Chroma.from_documents(pages, embedding=OpenAIEmbeddings())  # 임베딩 생성 및 저장
retriever = vectorstore.as_retriever()  # 검색기 생성

# **3. 검색된 문서 데이터를 문자열로 변환하는 함수**
def format_context(docs):
    """검색된 문서 리스트를 하나의 문자열로 병합"""
    return "\n".join([doc.page_content for doc in docs])

# **4. 프롬프트 템플릿 정의**
template = '''다음 컨텍스트를 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **5. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **6. RAG 체인 구성**
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}  # 컨텍스트와 질문 연결
    | prompt  # 프롬프트 적용
    | model   # 언어 모델 실행
    | StrOutputParser()  # 결과 파싱
)

# **7. 체인 실행**
retrieved_docs = retriever.get_relevant_documents("이 문서를 기반으로 발표 대본을 작성해주세요.")  # 문서 검색
formatted_context = format_context(retrieved_docs)  # 검색된 문서를 문자열로 변환

result = rag_chain.invoke({
    "context": formatted_context,  # 문자열 형태의 context 전달
    "question": "페이지별로 발표 대본을 작성해주세요."
})

# 결과 출력
print(result)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from PyPDF2 import PdfReader

# **1. PDF 텍스트 추출**
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"  # 각 페이지의 텍스트를 추가
    return text.strip()

# PDF 파일 경로 설정
pdf_path = "C:/wanted/Lang/Presentation-Agent/data/pdf/DeePrint.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

# **2. 프롬프트 템플릿 정의**
template = '''다음 PDF 내용을 기반으로 발표 대본을 작성하세요:
{context}

질문: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# **3. LLM 모델 초기화**
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# **4. 프롬프트 실행**
question = "페이지별로 발표 대본을 작성해주세요."
formatted_prompt = prompt.format(context=pdf_text, question=question)

result = model(formatted_prompt)

# 결과 출력
print(result)


In [ ]:
print(result.content)

In [12]:
import yt_dlp
from pydub import AudioSegment
import os

def time_convert(time_str):
    minutes = time_str // 100  
    seconds = time_str % 100   
    return minutes * 60 + seconds

def download_audio(url, save_dir, clip_idx, start=None, end=None):
    os.makedirs(save_dir, exist_ok=True)

    # 확장자 포함된 yt-dlp 저장용 임시 파일 경로
    temp_template = os.path.join(save_dir, f"karina_temp_{clip_idx}.%(ext)s")
    temp_wav = os.path.join(save_dir, f"karina_temp_{clip_idx}.wav")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': temp_template,
        'quiet': True,
        'force_ipv4': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 자르고 저장
    audio = AudioSegment.from_file(temp_wav, format="wav")
    if start and end:
        start_ms = time_convert(start) * 1000
        end_ms = time_convert(end) * 1000
        audio = audio[start_ms:end_ms]

    # 최종 파일명: cheo_1.wav, cheo_2.wav ...
    final_path = os.path.join(save_dir, f"karina_{clip_idx}.wav")
    audio.export(final_path, format="wav")

    os.remove(temp_wav)
    return audio


In [13]:
url = "https://www.youtube.com/watch?v=QXlm7RXDnMI&t=11s"
save_dir = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"
audio_data = []
start = 0
end = 20
i = 1 

for time in range(100,4000,1000):
    start += time
    end += time
    audio = download_audio(
        url = url,
        save_dir=save_dir,
        clip_idx=i,
        start=start,
        end=end
    )
    i += 1
    audio_data.append(audio)


In [ ]:
from zonos.model import Zonos

print("Zonos import 성공")


### Zonos

In [9]:
import os
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import torch._dynamo  # suppress warning if needed

# ❗ phonemizer용 환경변수 설정 (espeak.dll 대응)
os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

# ❗ torch compile 비활성화 (C++ 컴파일러 없이 실행)
torch._dynamo.config.suppress_errors = True

# 경로 설정
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"

# 모델 불러오기
device = "cuda"  # 또는 "cpu"
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)


c:\wanted\Lang\lang\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 경로 설정
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"

# 모델 불러오기
device = 'cuda' if torch.cuda.is_available() else 
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)


In [ ]:
import os
import torch
import torchaudio
import librosa
import numpy as np

# 입력 음성 로드
wav, sampling_rate = torchaudio.load(os.path.join(path, "karina_1.wav"))

# 스피커 임베딩 생성
speaker = model.make_speaker_embedding(wav, sampling_rate)

text = """
현대 사회에서 *AI* 기술은 빠르게 발전하고 있습니다.
우리의 '발표' 능력을 지원하는 시스템이 필요합니다.
"""

cond_dict = make_cond_dict(text=text, speaker=speaker, language="ko")
conditioning = model.prepare_conditioning(cond_dict)

codes = model.generate(conditioning, disable_torch_compile=True)
wavs = model.autoencoder.decode(codes).cpu()

# 원본 저장
torchaudio.save(os.path.join(path, "sample.wav"), wavs[0], model.autoencoder.sampling_rate)
print("🔊 sample.wav 생성 완료!")

# # 후처리
# sr = model.autoencoder.sampling_rate
# audio_np = wavs[0].squeeze().numpy()  # [1, N] → [N] 보장

# # 1.3배 빠르게
# y_fast = librosa.effects.time_stretch(audio_np, rate=1.3)

# # +2 반음
# y_shifted = librosa.effects.pitch_shift(y_fast, sr=sr, n_steps=2)

# # shape 조정
# if y_shifted.ndim == 1:
#     y_shifted = np.expand_dims(y_shifted, axis=0)  # → [1, samples]

# # numpy → torch
# y_tensor = torch.from_numpy(y_shifted).float()  # float32로 맞춤

# # 저장
# output_path = os.path.join(path, "fast_pitchup.wav")
# torchaudio.save(output_path, y_tensor, sr)

# print("⚡ 1.3배 빠르게 + 🎵 피치 +2반음 적용 완료!")
# print(f"📁 저장 위치: {output_path}")


Generating:  33%|███▎      | 855/2588 [00:23<00:48, 35.95it/s]


🔊 sample.wav 생성 완료!
⚡ 1.3배 빠르게 + 🎵 피치 +2반음 적용 완료!
📁 저장 위치: C:\wanted\Lang\Presentation-Agent\data\train_wav\fast_pitchup.wav


In [30]:
print("✅ 최종 y_tensor.shape:", y_tensor.shape)

✅ 최종 y_tensor.shape: torch.Size([1, 333588])


In [18]:

def zero_shot(path,file,text,name, model):
    torch.cuda.empty_cache()
    wav, sampling_rate = torchaudio.load(os.path.join(path, file))
    speaker = model.make_speaker_embedding(wav, sampling_rate)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker,
        language = "ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_zero_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    print("🔊 zero_shot.wav 생성 완료!")
    sr = model.autoencoder.sampling_rate
    audio_np = wavs[0].numpy()

    # [1] 1.3배 속도 (tempo)
    y_fast = librosa.effects.time_stretch(audio_np, rate=1.3)

    # [2] +2 반음 pitch up
    y_shifted = librosa.effects.pitch_shift(y_fast, sr=sr, n_steps=2)

    # 저장
    final_out_path = os.path.join(path, f"{name}_fast_pitchup.wav")
    sf.write(final_out_path, y_shifted, sr)


def few_shot(path, data, text, name, model):
    '''few-shot 보이스 클리닝'''
    torch.cuda.empty_cache()

    embeddings = []
    for file in data:
        wav, sampling_rate = torchaudio.load(os.path.join(path, file))
        emb = model.make_speaker_embedding(wav, sampling_rate)
        embeddings.append(emb)

    speaker_embedding = torch.stack(embeddings).mean(dim=0)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker_embedding,
        language="ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    # 음성 생성 (컴파일러 비활성화)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_few_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    print("🔊 few_shot.wav 생성 완료!")
    return codes


In [ ]:
path = r"C:\wanted\Lang\Presentation-Agent\data\train_wav"
file = "karina_2.wav"
data = [f"karina_{i}.wav" for i in range(1,5)]
name = "karina2"
text = """
이러한 문제를 해결하기 위해, AI 기반 발표 지원 시스템인 "저희 발표 안합니다!" 프로젝트가 기획되었습니다.
"""

zero_shot(path,file, text,name, model)
# few_shot(path,data, text,name)

In [ ]:
from transformers import pipeline
import scipy
synthesiser = pipeline("text-to-speech", "suno/bark")

speech = synthesiser("현대 사회에서 발표는 필수적인 활동입니다.", forward_params={"do_sample": True})

scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio
torch.cuda.empty_cache()

# download and load all models
preload_models(
    text_use_gpu=False,
    text_use_small=True,
    coarse_use_gpu=True,
    fine_use_gpu=True,
    codec_use_gpu=True
)


# generate audio from text
text_prompt = """현대 사회에서 발표는 필수적인 활동입니다.
하지만 많은 사람들은 내성적인 성격, 무대 공포증, 실수에 대한 두려움 등으로 인해 어려움을 겪습니다.
이러한 문제를 해결하기 위해, AI 기반 발표 지원 시스템인 "저희 발표 안합니다!" 프로젝트가 기획되었습니다."""
speech_array = generate_audio(text_prompt)

# play text in notebook
Audio(speech_array, rate=SAMPLE_RATE)


In [ ]:
# 📌 경로 설정 (MeloTTS 코드 추가)
import sys, os
sys.path.append("C:/wanted/Lang/MeloTTS")  # 너의 경로에 맞게 조정

# 📌 PyTorch 확인
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")


In [ ]:
import os

print(os.path.exists("C:/wanted/Lang/MeloTTS/melo/api.py"))
print(os.listdir("C:/wanted/Lang/MeloTTS"))


In [ ]:
import sys
sys.path.append("C:/wanted/Lang/MeloTTS/melo")  # ← melo 폴더 자체를 직접 추가

from api import TTS


In [ ]:
# 📌 Jupyter에서 재생 (IPython 오디오 위젯)
from IPython.display import Audio
Audio(output_path)

In [ ]:
from melo.api import TTS

# Speed is adjustable
speed = 1.0
device = 'cpu' # or cuda:0

text = "안녕하세요! 오늘은 날씨가 정말 좋네요."
model = TTS(language='KR', device=device)
speaker_ids = model.hps.data.spk2id

output_path = 'kr.wav'
model.tts_to_file(text, speaker_ids['KR'], output_path, speed=speed)

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from google.cloud import texttospeech

def ssml_to_audio(ssml_text: str) -> None:
    """
    SSML(Speech Synthesis Markup Language)을 기반으로 음성을 생성하여 MP3 파일로 저장하는 함수입니다.

    Args:
        ssml_text (str): SSML 형식의 문자열
    """

    # Text-to-Speech 클라이언트 객체 생성
    client = texttospeech.TextToSpeechClient()

    # SSML 텍스트를 음성 합성 입력으로 설정
    synthesis_input = texttospeech.SynthesisInput(ssml=ssml_text)

    # 사용할 목소리 설정
    # 언어는 영어("en-US"), 성별은 남성(MALE)으로 지정
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR",  # 영어 음성
        ssml_gender=texttospeech.SsmlVoiceGender.MALE  # 남성 목소리
    )

    # 출력할 오디오 형식 설정: MP3 파일
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # 음성 합성 요청 수행
    response = client.synthesize_speech(
        input=synthesis_input,   # SSML 기반 입력
        voice=voice,             # 목소리 설정
        audio_config=audio_config  # 오디오 설정
    )

    # 합성된 오디오 데이터를 MP3 파일로 저장
    with open("../../data/train_wav/test5_example.mp3", "wb") as out:
        out.write(response.audio_content)
        print("Audio content written to file " + "test_example.mp3")

In [35]:
test1 = """<speak> 안녕하세요오 만나서 반가워요오 나는 정재식입니다</speak>"""
test2 = """<speak> 발표하는 모델 <say-as interpret-as="characters">AI</say-as>오 인 용 입니다 만나서 반가워요</speak>"""
test3 = """<speak> 오늘 날씨 <mark name="오늘 날씨"/> 더럽게 <mark name="춥네요"/> 춥네요!!!!</speak>"""
test4 = """<speak> 
누구보다 빠르게 남들과는 다르게 색다르게 <prosody rate="fast" pitch="+2st">누구보다 빠르게 남들과는 다르게 색다르게</prosody>
<prosody rate="fast" pitch="+4st">누구보다 빠르게 남들과는 다르게 색다르게</prosody>
<prosody rate="fast" pitch="+6st">누구보다 빠르게 남들과는 다르게 색다르게</prosody>
<prosody rate="fast" pitch="+8st">누구보다 빠르게 남들과는 다르게 색다르게</prosody>
</speak>"""
test5 = """<speak> 
<emphasis level="strong">내가 그린 기린 그림은 잘 그린 기린 그림이고 니가 그린 기린 그림은 못 그린 기림 그림이다</emphasis>
</speak>"""
ssml_to_audio(test5)

Audio content written to file test_example.mp3


In [6]:
def synthesize_text(text):
    """입력된 문자열을 음성으로 합성하여 MP3 파일로 저장하는 함수입니다."""
    
    # Google Cloud의 Text-to-Speech 클라이언트 라이브러리를 import
    from google.cloud import texttospeech
    # Text-to-Speech API 클라이언트 객체 생성
    client = texttospeech.TextToSpeechClient()

    # 합성할 텍스트를 API 입력 형식에 맞게 변환
    input_text = texttospeech.SynthesisInput(text=text)

    # 사용할 음성 설정 (언어 코드만 지정하면 기본 한국어 음성이 사용됨)
    # 보다 세부적으로 설정하려면 voice name을 지정할 수도 있음
    voice = texttospeech.VoiceSelectionParams(
        language_code="ko-KR",  # 한국어(Korean) 설정
        name = "ko-KR-Chirp3-HD-Leda"
    )

    # 오디오 출력 형식 설정 (여기서는 MP3로 설정)
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # 실제로 음성 합성을 요청하는 부분
    response = client.synthesize_speech(
        input=input_text,        # 입력 텍스트
        voice=voice,             # 음성 설정
        audio_config=audio_config  # 오디오 포맷 설정
    )

    # 응답의 오디오 콘텐츠(audio_content)는 바이너리 형식으로 전달됨
    # MP3 파일로 저장
    with open("../../data/train_wav/output.mp3", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')  # 완료 메시지 출력


In [7]:
text = """
이전 슬라이드에서는 "발표 안 합니다!" 서비스 구현에 사용된 핵심 기술 스택 – FastAPI, Streamlit, LangChain, Ollama, Hugging Face, ChromaDB, OpenAI – 에 대해 자세히 알아보았습니다. 이러한 최첨단 기술들이 어떻게 조화롭게 작동하여 강력한 발표 자동화 시스템을 구축하는지 기억하시죠? 이번 슬라이드에서는 저희 프로젝트의 미래, 즉 "발표 안 합니다!" 서비스의 발전 방향에 대해 이야기해보겠습니다.  프로젝트 방향성 03 슬라이드에서 확인할 수 있듯이, 저희는 단순히 현재에 안주하지 않고 끊임없이 발전하는 서비스를 제공하고자 합니다.
현재 "발표 안 합니다!"는  발표 자료를 기반으로 대본을 생성하고 음성 합성을 통해 발표를 진행하는 핵심 기능을 제공합니다. 하지만 저희는 여기서 멈추지 않고, 사용자에게 더욱 자연스럽고 풍부한 발표 경험을 제공하기 위해 몇 가지 핵심 기능들을 추가할 계획입니다.  실시간 상호작용 기능을 통해 청중의 질문에 즉각적으로 답변하고,  마치 사람처럼 자연스럽게 말하는 기능을 구현하여 발표의 몰입도를 높일 것입니다. 또한, 사용자별 맞춤 발표 스타일 적용 기능을 통해 개인의 발표 스타일에 맞춘 최적화된 대본을 제공하고, 디지털 아바타를 활용한 발표 기능을 추가하여 시각적인 요소까지 더욱 풍부하게 만들어갈 예정입니다.
더 나아가, 현재는 일반적인 발표 자료에 초점을 맞추고 있지만,  향후에는 특정 도메인에 특화된 발표 대본 생성 기능을 추가하여 다양한 분야에서 활용될 수 있도록 확장할 계획입니다.  이러한 발전 방향을 통해 "발표 안 합니다!"는 단순한 발표 도구를 넘어,  누구나 쉽고 효과적으로 자신의 아이디어를 전달하고 소통할 수 있도록 돕는  혁신적인 플랫폼으로 성장할 것입니다.
"""
synthesize_text(text)

Audio content written to file "output.mp3"


In [7]:
# gcloud ml speech voices list

In [8]:
from google.cloud import texttospeech

client = texttospeech.TextToSpeechClient()

ssml_text = """
<speak>
    오늘은 <emphasis level="strong">중요한 발표</emphasis>가 있습니다.
    <break time="500ms"/> 모두 집중해 주세요.
</speak>
"""

synthesis_input = texttospeech.SynthesisInput(ssml=ssml_text)

voice = texttospeech.VoiceSelectionParams(
    language_code="ko-KR",
    name="ko-KR-Wavenet-B"
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

response = client.synthesize_speech(
    input=synthesis_input,
    voice=voice,
    audio_config=audio_config
)

with open("output.mp3", "wb") as out:
    out.write(response.audio_content)
